<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Dash - Create Plot Based on OWID's oil consumption data
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Dash/Dash_Create_Plot_based_on_OWID's_oil_consumption_data.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/open_in_naas.svg"/></a><br><br><a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=&template=template-request.md&title=Tool+-+Action+of+the+notebook+">Template request</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=Dash+-+Create+Interactive+Plot:+Error+short+description">Bug report</a> | <a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/Naas/Naas_Start_data_product.ipynb" target="_parent">Generate Data Product</a>

**Tags:** #dash #dashboard #plotly #naas #asset #analytics #dropdown #callback #bootstrap #snippet

**Author:** [Zihui Ouyang](https://www.linkedin.com/in/zihui-ouyang-539626227/)

**Description:** This notebook creates an interactive plot using Dash app infrastructure with OWID's oil consumption data.

**References:**
- https://stackoverflow.com/questions/70886359/dash-python-making-subplots-when-multiple-parameters-are-selected
- https://dash-example-index.herokuapp.com/line-charts

## Input

In [ ]:
try:
    import dash
except:
    !pip install dash --user
    import dash
try:
    import dash_bootstrap_components as dbc
except:
    !pip install dash_bootstrap_components --user
    import dash_bootstrap_components as dbc
import pandas as pd
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px

### Setup Variables
- `DASH_PORT`: specify a port number for Dash
- `url`: URL to get data from Excel

In [ ]:
DASH_PORT = 8050
url = "https://www.bp.com/content/dam/bp/business-sites/en/global/corporate/xlsx/energy-economics/statistical-review/bp-stats-review-2022-all-data.xlsx"

## Model

### Initialize Dash App

In [ ]:
app = dash.Dash(
    requests_pathname_prefix=f'/user/{os.environ.get("JUPYTERHUB_USER")}/proxy/{DASH_PORT}/',
    external_stylesheets=[dbc.themes.BOOTSTRAP],
    meta_tags=[
        {"name": "viewport", "content": "width=device-width, initial-scale=1.0"}
    ],
)

# app = dash.Dash() if you are not in Naas

### Get Data

In [ ]:
contents_df = pd.read_excel(url, sheet_name = "Oil Consumption - Barrels", header = 2)
contents_df = contents_df.drop([0,5,18,54,64,86,106,111,112,113,114,115,116,117,118]) # Drop unnecessary columns
contents_df = contents_df.reset_index(drop=True)
entity = []  # Lists to create a new dataframe
year = []
oil_consumption = []
for i in range(len(contents_df)):
    for j in range(57):
        entity.append(contents_df.iloc[i,0])
        year.append(1965+j)
        oil_consumption.append(contents_df.iloc[i,j+1])
new_dict = {"Entity": entity,
           "Year": year,
           "oil_consumption": oil_consumption}
new_df = pd.DataFrame(data=new_dict)

### Create Dash App

In [ ]:
p = "oil_consumption"
app.layout = html.Div(
    [
        html.H4("Oil consumption from 1965 to 2021"),
        html.P("Select countries"),
        dcc.Dropdown(
            id="countries",
            options=new_df.Entity.unique(),
            value=["US"],
            multi = True
        ),
        dcc.Graph(id=p, figure={}, style={'display': 'none'}) 
    ]
)

@callback(
    Output(p, 'figure'),
    Output(p, 'style'),
    Input('countries', 'value')
)

def update_graph(country):
    dff = new_df.Entity.isin(country)
    figures = px.line(new_df[dff], x='Year', y=p, color = "Entity")
    styles = {'display': 'block'}   
    return figures, styles


## Output

### Generate URL and show logs

In [ ]:
if __name__ == "__main__":
    app.run_server(proxy=f"http://127.0.0.1:{DASH_PORT}::https://app.naas.ai")